In [1]:
import numpy as np
import pandas as pd
import Data as df

18 unique words found


In [18]:
class RNN:
    def __init__(self,vocab_len,output_len,hidden_size =31):
        self.Wya = np.random.randn(output_len,hidden_size)/1000
        self.Wax = np.random.randn(hidden_size,vocab_len)/1000
        self.Waa = np.random.randn(hidden_size,hidden_size)/1000

        self.by = np.zeros((hidden_size,1))
        self.ba = np.zeros((output_len,1))
        self.last_as = []

    def forward_prop(self,sentences):
        a = 0
        for i ,x in enumerate(sentences):
            # Generate/Update a with the inputs
            a = np.tanh(np.matmul(self.Wax,x) + np.matmul(self.Waa,a) + self.ba)
            # Store the a for backprop
            self.last_as[i+1] = { 0: a }

        self.last_sentences = sentences
        #Store the inputs for backprop

        y = np.matmul(self.Wya,a) + self.by
        
        return y, a


    def back_prop(self,dL_dy,learning_rate = 0.001):
        dWya =  np.zeros(self.Wya.shape)
        dWax =  np.zeros(self.Wax.shape)
        dWaa =  np.zeros(self.Waa.shape)
        dby  =  np.zeros(self.by.shape)
        dba  =  np.zeros(self.ba.shape)

        n = len(self.last_sentences)

        dWya = np.matmul(dL_dy,self.last_as[n])
        dby = dL_dy

        temp = 1
        dh = self.Wya

        for i in reversed(range(n)):
            temp = dh*(1-self.last_as[i+1]**2)
            
            dWaa =+ temp*self.last_as[i]
            dWax =+ temp*self.last_sentences[i]
            dba  =+ temp
            dh =self.Waa*temp

        self.Wax = self.Wax  - learning_rate*dWax
        self.Waa = self.Waa  - learning_rate*dWaa
        self.Wya = self.Wya  - learning_rate*dWya

        self.by =  self.by -learning_rate*dby
        self.ba =  self.ba -learning_rate*dba 

In [16]:
df.vocab_size = len(df.word_to_idx)
rnn = RNN(df.vocab_size, 1)

In [5]:
def softmax(xs):
  # Applies the Softmax Function to the input array.
  return np.exp(xs) / sum(np.exp(xs))

In [20]:
# Initialize our RNN!
rnn = RNN(df.vocab_size, 2)

def processData(data, backprop=True):
  '''
  Returns the RNN's loss and accuracy for the given data.
  - data is a dictionary mapping text to True or False.
  - backprop determines if the backward phase should be run.
  '''
  items = list(data.items())
  np.random.shuffle(items)

  loss = 0
  num_correct = 0

  for x, y in items:
    inputs = df.createInputs(x)
    target = int(y)

    # Forward
    out, _ = rnn.forward_prop(inputs)
    probs = softmax(out)

    # Calculate loss / accuracy
    loss -= np.log(probs)
    num_correct += int(np.argmax(probs) == target)

    if backprop:
      # Build dL/dy
      d_L_d_y = probs
      
      d_L_d_y = target - d_L_d_y 
      #Softmax derivative

      # Backward
      rnn.back_prop(d_L_d_y)

  return loss / len(data), num_correct / len(data)

# Training loop
for epoch in range(1000):
  train_loss, train_acc = processData(df.train_data)

  if epoch % 100 == 99:
    print('--- Epoch %d' % (epoch + 1))
    print('Train:\tLoss %.3f | Accuracy: %.3f' % (train_loss, train_acc))

    test_loss, test_acc = processData(df.test_data, backprop=False)
    print('Test:\tLoss %.3f | Accuracy: %.3f' % (test_loss, test_acc))

ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)